##### Feature pipeline
- Register the arranged data to hopsworks 

#### Training pipeline
- read it from hopsworks in training pipeline and split into test train
- train the model
- evaluate the model
- register to hopsworks again

#### Daily inference pipeline
- get last data and put to hopsworks

#### UI
- get the recent data from hopsworks
- get user input (how many days the prediction will be)
- predict and plot  


In [ ]:
import configparser
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np
import datetime as dt
import hopsworks

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login(api_key_value=api_key, project="project0")
fs = project.get_feature_store()

In [ ]:
#read configs
config = configparser.ConfigParser()
config_file_path = "C:/Users/User/Desktop/mldl/project/config.ini"  # Replace with the actual path to your config.ini file
config.read(config_file_path)
config.read("config.ini") #The Config.ini file we just created

In [ ]:
api_key = config["binance"]["api_key"]
api_secret = config["binance"]["api_secret"]

In [ ]:
client = Client(api_key, api_secret)

In [ ]:
tickers = client.get_all_tickers()
tickers #This is used to get the entire tickets available on Binance

In [ ]:
#Wrap the data into a pandas dataFrame
binance_df = pd.DataFrame(tickers)
binance_df.head()

In [ ]:
#Let make the Symbol the Index value
binance_df.set_index("symbol", inplace=True)
binance_df

In [ ]:
float(binance_df.loc["BTCUSDT"]["price"])

In [ ]:
depth = client.get_order_book(symbol="BTCUSDT")
depth

In [ ]:
depth_df = pd.DataFrame(depth['bids'])
depth_df.columns = ["Price","Volume"]
depth_df.head()

**Get historical data**

In [ ]:
historical_data = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "1 Jan 2018")
historical_data

In [ ]:
historical_df = pd.DataFrame(historical_data)


historical_df.columns = ["Open time","Open","High","Low","Close","Volume","Close time","Quote asset volume","Number of trades",
                   "Taker buy base asset volume","Taker buy quote asset volume","Ignore"]
                  
historical_df.head() 

In [ ]:
historical_df.info()

**Data Transformation & Preprocessing**

In [ ]:
#We need to convert the Time from Object to datetime format
historical_df["Open time"] = pd.to_datetime(historical_df["Open time"]/1000, unit="s")
historical_df["Close time"] = pd.to_datetime(historical_df["Close time"]/1000, unit="s")

In [ ]:
#Covert others to float datatype
numeric_columns = ["Open","High","Low","Close","Volume","Quote asset volume",
                   "Taker buy base asset volume","Taker buy quote asset volume","Ignore"]

In [ ]:
historical_df[numeric_columns] = historical_df[numeric_columns].apply(pd.to_numeric, axis=1)

In [ ]:
historical_df.dtypes

In [ ]:
historical_df.describe() # This used to get the statistical analysis

**Data Visualization**

In [ ]:
import mplfinance as mpf 
import matplotlib.pyplot as plt

In [ ]:
mpf.plot(historical_df.set_index("Close time").tail(200))

In [ ]:
mpf.plot(historical_df.set_index("Close time").tail(200),type="candle", style="charles", volume=True,title="BTCUSDT Last 200 Days",
    mav=(10,20,30))

**Save features to hopsworks**

In [ ]:
historical_df = historical_df.drop('Ignore', axis=1)
historical_df.columns = historical_df.columns.str.lower().str.replace(' ', '_')
historical_df

In [ ]:
btc_fg = fs.get_or_create_feature_group(
    name="btc",
    version=1,
    primary_key=["open","high","low","close","volume","quote_asset_volume",
                   "taker_buy_base_asset_volume","taker_buy_quote_asset_volume"], 
    description="bitcoin historical data since january 2018")
btc_fg.insert(historical_df)